In [6]:
import pandas as pd
import numpy as np
import random
import time

# Definindo as temporadas e equipes
seasons = [str(season) for season in range(2023, 2025)]teams = ['OAK', 'TOR', 'KCR', 'SFG', 'MIA', 'TBR', 'BOS', 'CLE', 'CHC', 'NYY',
         'CHW', 'LAA', 'BAL', 'TEX', 'CIN', 'SEA', 'COL', 'HOU', 'ATL', 'DET',
         'ARI', 'MIN', 'PIT', 'MIL', 'PHI', 'LAD', 'SDP', 'NYM', 'WSN', 'STL']

# Inicializando uma lista para armazenar os resultados
results = []

# Inicializando um DataFrame vazio para coletar os dados
batting_team_df = pd.DataFrame()

# Coletando dados para todas as equipes
for season in seasons:
    for team in teams:
        url = f"https://www.baseball-reference.com/teams/{team}/{season}.shtml#all_team_batting"
        print(f"Coletando dados de: {url}")
        bt_df = pd.read_html(url, header=0, attrs={'id': 'team_batting'})[0]
        bt_df.insert(loc=0, column='Season', value=season)
        bt_df.insert(loc=1, column='Team', value=team)
        batting_team_df = pd.concat([batting_team_df, bt_df], ignore_index=True)
        time.sleep(random.randint(7, 8))

# Convertendo colunas relevantes para numérico, tratando erros
batting_team_df['SO'] = pd.to_numeric(batting_team_df['SO'], errors='coerce')
batting_team_df['R'] = pd.to_numeric(batting_team_df['R'], errors='coerce')

# Filtrando e processando os dados
for team in teams:
    # Filtrando os dados do time
    batting_data = batting_team_df[batting_team_df['Team'] == team]

    # Filtrando as temporadas
    data_2023 = batting_data[batting_data['Season'] == '2023']
    data_2024 = batting_data[batting_data['Season'] == '2024']
    
    # Calculando a %K ponderada
    total_SO_2023 = data_2023['SO'].sum()
    total_SO_2024 = data_2024['SO'].sum()
    total_AB_2023 = data_2023['R'].sum() + data_2023['SO'].sum()  # Simulando oportunidades de strikeout
    total_AB_2024 = data_2024['R'].sum() + data_2024['SO'].sum()  # Simulando oportunidades de strikeout

    # Calculando %K
    if total_AB_2023 > 0 and total_AB_2024 > 0:
        k_percentage_2023 = total_SO_2023 / total_AB_2023
        k_percentage_2024 = total_SO_2024 / total_AB_2024
        weighted_K = (0.25 * k_percentage_2023) + (0.75 * k_percentage_2024)
    else:
        weighted_K = 0  # Caso não haja oportunidades

    # Adicionando os resultados à lista
    results.append({'Team': team, '%K': weighted_K})

# Convertendo a lista de resultados em um DataFrame
results_df = pd.DataFrame(results)

# Salvando os resultados em um novo arquivo CSV
results_df.to_csv('team_strikeout_percentage.csv', index=False)

print("Previsão da %K dos times foi salva com sucesso!")


Coletando dados de: https://www.baseball-reference.com/teams/OAK/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/TOR/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/KCR/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/SFG/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/MIA/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/TBR/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/BOS/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/CLE/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/CHC/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/NYY/2023.shtml#all_team_batting
Coletando dados de: https://www.baseball-reference.com/teams/CHW/2023.